In [34]:
%cd C:\Users\Academic\Documents\Projects\sales-performance-dashboard

import pandas as pd
from sqlalchemy import create_engine

df_cleaned = pd.read_csv("data/processed/superstore_cleaned.csv", parse_dates=["Order Date", "Ship Date"])

engine = create_engine("sqlite:///data/superstore.db", echo=True)

df_cleaned.to_sql("raw_sales", engine, if_exists="replace", index=False)

#Importing libaries, dataset,


C:\Users\Academic\Documents\Projects\sales-performance-dashboard
2025-05-26 15:13:40,271 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 15:13:40,291 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw_sales")
2025-05-26 15:13:40,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 15:13:40,296 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw_sales")
2025-05-26 15:13:40,298 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 15:13:40,300 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-05-26 15:13:40,301 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 15:13:40,303 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-05-26 15:13:40,304 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 15:13:40,307 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("raw_sales")
2025-05-

9994

In [36]:
from sqlalchemy import text

with engine.begin() as conn:

    conn.execute(text(
    
"""
CREATE TABLE IF NOT EXISTS dim_date AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales """))



2025-05-26 15:18:31,877 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 15:18:31,878 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS dim_date AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales 
2025-05-26 15:18:31,879 INFO sqlalchemy.engine.Engine [cached since 281.3s ago] ()
2025-05-26 15:18:31,880 INFO sqlalchemy.engine.Engine COMMIT
2025-05-26 15:18:31,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 15:18:31,882 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS dim_produ

In [ ]:
with engine.begin() as conn:
    conn.execute(text(

"""
CREATE TABLE IF NOT EXISTS dim_product AS
    SELECT DISTINCT
      Category,
      "Sub-Category" AS sub_category

FROM raw_sales
   """         
                   )
                )


In [ ]:
with engine.begin() as conn:
    conn.execute(text(
"""
CREATE TABLE IF NOT EXISTS dim_region AS 
    SELECT DISTINCT
      Region
      
FROM raw_sales
  """         
                   )
                )

In [ ]:
category_data_sql = text("""

WITH 
    category_profits AS (

    SELECT
    
        CATEGORY       AS category,
        SUM(Profit)    AS category_profit
        
    FROM raw_sales
    GROUP BY Category   ),  

    
    top_categories AS (
    
     SELECT
        cp1.category,
        cp1.category_profit
   
     FROM category_profits cp1
     
     WHERE 
       (
        SELECT COUNT(*)
        FROM category_profits cp2
        WHERE cp2.category_profit > cp1.category_profit) < 3 
    ),
    
    
    product_profits AS (

     SELECT
        Category       AS category,
        "Product Name" AS product,
        SUM(Profit)    AS product_profit
        
     FROM raw_sales
     GROUP BY Category, "Product Name"
     )
    
    SELECT 
     tc.category AS "Category",
     tc.category_profit AS "Category Profit",
     pp.product AS "Product Name",
     pp.product_profit AS "Product Profit"

    FROM top_categories tc
    
    JOIN product_profits pp
        ON pp.category = tc.category
    
    WHERE (
        SELECT COUNT(*)
        FROM product_profits pp2
        WHERE pp2.category = pp.category
          AND pp2.product_profit > pp.product_profit) < 5
    
    ORDER BY
        tc.category_profit DESC,
        pp.product_profit DESC;

    """)

In [ ]:
category_data = pd.read_sql_query(category_data_sql, engine)


In [ ]:
category_data.to_csv("data/processed/category_data.csv", index=False)